# Data collection
Gather public SPP Weis data from https://marketplace.spp.org/groups/operational-data-weis

In [ ]:
import os
import pandas as pd
import duckdb

import requests
from io import StringIO

import ibis
import ibis.selectors as s
ibis.options.interactive = True

# logging
import logging

In [ ]:
# define log
logging.basicConfig(level=logging.INFO)
log = logging.getLogger(__name__)


In [ ]:
os.chdir('../..')

In [ ]:
import src.data_collection as dc

## Set up backfill parameters

In [ ]:
start_date = '2024-09-15'
end_date = '2024-09-28'
num_day_step = 1
day_list = pd.date_range(start = start_date, end=end_date, freq=f'{num_day_step }d')
day_list = [pd.Timestamp(d) for d in day_list]
day_list

## Mid Term Load Forecast

![_](../../imgs/mtlf.PNG)

HOUR = {0000, ..., 2300}
DAY = {01, ..., 31}

In [ ]:
for d in day_list:
    dc.collect_upsert_mtlf(end_ts=d, n_periods=24*num_day_step+1, backfill=True)

## Mid Term Resource Forecast

![_](../../imgs/mtrf.PNG)

In [ ]:
for d in day_list:
    dc.collect_upsert_mtrf(end_ts=d, n_periods=24*num_day_step+1, backfill=True)

## LMP settlement location prices

![_](../../imgs/lmp_settlement_location.PNG)

## LMP upsert

### LMP daily file

In [ ]:
for d in day_list:
    dc.collect_upsert_lmp(end_ts=d, n_periods=num_day_step+1, daily_file=True)